In [13]:
import sys
sys.path.append('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/')
import pandas as pd 
import numpy as np 
import datasets
from datasets import Dataset as TFDataset 
import pickle
from bm25.rank_bm25 import BM25Okapi
import random
# import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
from transformers import PreTrainedTokenizer, AutoTokenizer
import multiprocessing

In [14]:
def min_max_normalization(score_df, score_name, use_zero=True):
    # score_df_g = cudf.from_pandas(score_df)
    if not use_zero:
        score_df_ = score_df[score_df[score_name] != 0]
    else:
        score_df_ = score_df
    scores_min = score_df_[['sess_id', score_name]].groupby('sess_id')[score_name].min().to_numpy()
    scores_max = score_df_[['sess_id', score_name]].groupby('sess_id')[score_name].max().to_numpy()

    score_df['scores_min'] = scores_min[score_df['sess_id']]
    score_df['scores_max'] = scores_max[score_df['sess_id']]

    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df['min_max_'+score_name] = (score_df[score_name] - score_df['scores_min']) / (score_df['scores_max'] - score_df['scores_min'])
    score_df.drop(columns=['scores_min', 'scores_max'], inplace=True)

    if not use_zero:
        score_df['min_max_'+score_name][score_df[score_name] == 0] = 0.0


    # del scores_sum_g
    # del merged_score_df_g 

In [15]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

# Merge test scores

In [16]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_test_feature.parquet'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'

In [17]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)


In [19]:
merged_candidates_feature_test = read_merged_candidates_feature_test()

In [21]:
min_max_score_list = ['title_BM25_scores', 'desc_BM25_scores', 'feat_BM25_scores', 'title_bert_scores', 'text_bert_scores']
# min_max_score_list = ['feat_BM25_scores']

In [22]:
for score_name in min_max_score_list:
    score_df = merged_candidates_feature_test[['sess_id', score_name]]
    min_max_normalization(score_df, score_name, use_zero=True)
    merged_candidates_feature_test['min_max_'+score_name] = score_df['min_max_'+score_name]

/tmp/ipykernel_377317/1800093031.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['scores_min'] = scores_min[score_df['sess_id']]
/tmp/ipykernel_377317/1800093031.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['scores_max'] = scores_max[score_df['sess_id']]
/tmp/ipykernel_377317/1800093031.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [24]:
score_name_list = ['min_max_'+score_name for score_name in min_max_score_list]
cast_dtype(merged_candidates_feature_test, score_name_list)
merged_candidates_feature_test.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [27]:
merged_candidates_feature_test['min_max_feat_BM25_scores'], merged_candidates_feature_test['feat_BM25_scores']

(0           0.000000
 1           0.000000
 2           0.087639
 3           0.015446
 4           0.487077
               ...   
 96556030    0.000000
 96556031    0.000000
 96556032    0.000000
 96556033    0.115995
 96556034    0.000000
 Name: min_max_feat_BM25_scores, Length: 96556035, dtype: float32,
 0            0.000000
 1            0.000000
 2            9.070796
 3            1.598722
 4           50.413322
               ...    
 96556030     0.000000
 96556031     0.000000
 96556032     0.000000
 96556033    19.868626
 96556034     0.000000
 Name: feat_BM25_scores, Length: 96556035, dtype: float32)

In [28]:
merged_candidates_feature_test['min_max_title_BM25_scores']

0           0.000000
1           0.000000
2           0.000000
3           0.000000
4           0.309716
              ...   
96556030    0.372690
96556031    0.000000
96556032    0.000000
96556033    0.265195
96556034    0.345440
Name: min_max_title_BM25_scores, Length: 96556035, dtype: float32

In [29]:
merged_candidates_feature_test['desc_BM25_scores']

0            0.000000
1            3.694895
2           22.245871
3           18.249533
4           73.129723
              ...    
96556030    70.941124
96556031     0.000000
96556032     0.000000
96556033    31.416847
96556034     0.000000
Name: desc_BM25_scores, Length: 96556035, dtype: float32

In [30]:
merged_candidates_feature_test['min_max_desc_BM25_scores']

0           0.000000
1           0.010312
2           0.062083
3           0.050930
4           0.204088
              ...   
96556030    0.114051
96556031    0.000000
96556032    0.000000
96556033    0.050508
96556034    0.000000
Name: min_max_desc_BM25_scores, Length: 96556035, dtype: float32

In [31]:
merged_candidates_feature_test['text_bert_scores'] 

0             0.000000
1           382.443390
2           376.119781
3           366.794250
4           377.558044
               ...    
96556030    421.320526
96556031      0.000000
96556032    410.857330
96556033    418.673431
96556034    421.208649
Name: text_bert_scores, Length: 96556035, dtype: float32

In [32]:
merged_candidates_feature_test['min_max_text_bert_scores'] 

0           0.000000
1           0.826050
2           0.683499
3           0.473276
4           0.715921
              ...   
96556030    0.771414
96556031    0.000000
96556032    0.211611
96556033    0.629789
96556034    0.765429
Name: min_max_text_bert_scores, Length: 96556035, dtype: float32

In [26]:
merged_candidates_min_max_BM25_Bert_test = merged_candidates_feature_test[['sess_id', 'product',
                                                                           'min_max_title_BM25_scores', 'min_max_desc_BM25_scores', 'min_max_feat_BM25_scores', 
                                                                           'min_max_title_bert_scores', 'min_max_text_bert_scores']]

In [33]:
merged_candidates_min_max_BM25_Bert_test

,sess_id,product,min_max_title_BM25_scores,min_max_desc_BM25_scores,min_max_feat_BM25_scores,min_max_title_bert_scores,min_max_text_bert_scores
0,0,B000Q87D0Q,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,B000QB30DW,0.000000,0.010312,0.000000,0.726431,0.826050
2,0,B004BIG55Q,0.000000,0.062083,0.087639,0.531170,0.683499
3,0,B0053FTNQY,0.000000,0.050930,0.015446,0.350456,0.473276
4,0,B007QWII1S,0.309716,0.204088,0.487077,0.676267,0.715921
...,...,...,...,...,...,...,...
96556030,316971,B0B82N3CQQ,0.372690,0.114051,0.000000,0.694524,0.771414
96556031,316971,B0BB9NW3F3,0.000000,0.000000,0.000000,0.000000,0.000000
96556032,316971,B0BDMVKTQ3,0.000000,0.000000,0.000000,0.175180,0.211611
96556033,316971,B0BHW1D5VP,0.265195,0.050508,0.115995,0.610117,0.629789


In [ ]:
merged_candidates_min_max_BM25_Bert_test.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/transpose/cxl_merged_candidates_min_max_BM25_Bert_scores_test.parquet')